#### ANSWER 7


In [2]:
from keras.datasets import mnist

#### ANSWER 8



In [ ]:
import numpy as np
import wandb
from keras.datasets import fashion_mnist


# Neural Network Class: feed_forward_NN_4

class feed_forward_NN_8:
    def __init__(self,
                 layers,
                 optimizer,
                 learning_rate,
                 momentum,
                 beta1,
                 beta2,
                 beta,
                 epsilon,
                 weight_decay,
                 init_type,
                 activation,
                 loss_func
                 ):
    
        
        self.layers = layers
        self.layer_n = len(layers)
        self.optimizer = optimizer.lower()
        self.lr = learning_rate
        self.momentum = momentum
        self.beta1 = beta1
        self.beta2 = beta2
        self.beta = beta
        self.epsilon = epsilon
        self.weight_decay = weight_decay
        self.init_type = init_type.lower()
        self.activation = activation.lower()
        self.loss_func=loss_func.lower()

        # Initialize Weights & BiaseS
        self.weights = []
        self.biases = []
        for i in range(self.layer_n - 1):
            if self.init_type == "xavier":
                # "Xavier" initialization
                w = np.random.randn(layers[i], layers[i+1]) * np.sqrt(1.0 / layers[i])
            else:
                # "random" initialization
                w = np.random.randn(layers[i], layers[i+1]) * np.sqrt(2.0 / layers[i])
            b = np.zeros((1, layers[i+1]))
            self.weights.append(w)
            self.biases.append(b)

        # initialize extra Params 
        if self.optimizer in ["momentum", "nesterov", "rmsprop", "adam", "nadam"]:
            self.v_w = [np.zeros_like(w) for w in self.weights]
            self.v_b = [np.zeros_like(b) for b in self.biases]
        if self.optimizer in ["adam", "nadam"]:
            self.m_w = [np.zeros_like(w) for w in self.weights]
            self.m_b = [np.zeros_like(b) for b in self.biases]
            self.t = 0

    # activations 
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def tanh(self, x):
        return np.tanh(x)
    
    def relu(self, x):
        return np.maximum(0, x)

    def activate(self, x):
        if self.activation == "sigmoid":
            return self.sigmoid(x)
        elif self.activation == "tanh":
            return self.tanh(x)
        elif self.activation == "relu":
            return self.relu(x)
        else:
            return self.sigmoid(x) 
        
    # derivatives
    def derivative(self, a):

        if self.activation == "sigmoid":
            return a * (1 - a)
        elif self.activation == "tanh":
            return 1 - a**2
        elif self.activation == "relu":
            return (a > 0).astype(float)
        else:
            return a * (1 - a) 

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    # Forward Pass
    def forward_pass(self, x):
        self.h = [x]  
        # Hidden layers
        for i in range(self.layer_n - 2):
            z = np.dot(self.h[i], self.weights[i]) + self.biases[i]
            act = self.activate(z)
            self.h.append(act)
        # Output layer- softmax
        z_out = np.dot(self.h[-1], self.weights[-1]) + self.biases[-1]
        out = self.softmax(z_out)
        self.h.append(out)
        return self.h

    # Backward Pass
    def backward_prop(self, y_true):
        m = y_true.shape[0]
        dw = [None] * (self.layer_n - 1)
        db = [None] * (self.layer_n - 1)

        # Cross-entropy derivative for output layer
        if self.loss_func=="cross_entropy":
            delta = self.h[-1] - y_true  # shape: (batch_size, output_dim)
        elif self.loss_func=="sq_error":
            batch_size_sq=len(self.h[-1])
            classes_sq=len(self.h[-1][0])
            delta=np.zeros((batch_size_sq,classes_sq))

            for i in range(batch_size_sq):
                jacobian_softmax= np.diag(self.h[-1][i]) - np.outer(self.h[-1][i], self.h[-1][i])
                # print(jacobian_softmax.shape)
                # print(self.h[-1][i])    
                delta[i]= 2*np.dot(self.h[-1][i]-y_true[i], jacobian_softmax)
                
        else:
            delta = self.h[-1] - y_true 


        # Propagation
        for i in reversed(range(self.layer_n - 1)):
            dw[i] = np.dot(self.h[i].T, delta) / m
            db[i] = np.sum(delta, axis=0, keepdims=True) / m
            if i > 0:
                # For hidden layers, multiply by derivative of activation
                delta = np.dot(delta, self.weights[i].T) * self.derivative(self.h[i])
        return dw, db

    # Param Updates for "Non-Nesterov" 
    def _update_params(self, dw, db):
        # Add weight decay to each gradient
        for i in range(self.layer_n - 1):
            dw[i] += self.weight_decay * self.weights[i]

        if self.optimizer == "sgd":
            for i in range(self.layer_n - 1):
                self.weights[i] -= self.lr * dw[i]
                self.biases[i] -= self.lr * db[i]

        elif self.optimizer == "momentum":
            for i in range(self.layer_n - 1):
                self.v_w[i] = self.momentum * self.v_w[i] + dw[i]
                self.v_b[i] = self.momentum * self.v_b[i] + db[i]
                self.weights[i] -= self.lr * self.v_w[i]
                self.biases[i] -= self.lr * self.v_b[i]

        elif self.optimizer == "rmsprop":
            for i in range(self.layer_n - 1):
                self.v_w[i] = self.beta* self.v_w[i] + (1 - self.beta) * (dw[i] ** 2)
                self.v_b[i] = self.beta * self.v_b[i] + (1 - self.beta) * (db[i] ** 2)
                self.weights[i] -= self.lr * dw[i] / (np.sqrt(self.v_w[i]) + self.epsilon)
                self.biases[i]  -= self.lr * db[i] / (np.sqrt(self.v_b[i]) + self.epsilon)

        elif self.optimizer == "adam":
            self.t += 1
            for i in range(self.layer_n - 1):
                self.m_w[i] = self.beta1 * self.m_w[i] + (1 - self.beta1) * dw[i]
                self.m_b[i] = self.beta1 * self.m_b[i] + (1 - self.beta1) * db[i]
                self.v_w[i] = self.beta2 * self.v_w[i] + (1 - self.beta2) * (dw[i] ** 2)
                self.v_b[i] = self.beta2 * self.v_b[i] + (1 - self.beta2) * (db[i] ** 2)

                # bias correction
                m_w_hat = self.m_w[i] / (1 - self.beta1 ** self.t)
                m_b_hat = self.m_b[i] / (1 - self.beta1 ** self.t)
                v_w_hat = self.v_w[i] / (1 - self.beta2 ** self.t)
                v_b_hat = self.v_b[i] / (1 - self.beta2 ** self.t)

                self.weights[i] -= self.lr * m_w_hat / (np.sqrt(v_w_hat) + self.epsilon)
                self.biases[i]  -= self.lr * m_b_hat / (np.sqrt(v_b_hat) + self.epsilon)

        elif self.optimizer == "nadam":
            self.t += 1
            for i in range(self.layer_n - 1):
                self.m_w[i] = self.beta1 * self.m_w[i] + (1 - self.beta1) * dw[i]
                self.m_b[i] = self.beta1 * self.m_b[i] + (1 - self.beta1) * db[i]
                self.v_w[i] = self.beta2 * self.v_w[i] + (1 - self.beta2) * (dw[i] ** 2)
                self.v_b[i] = self.beta2 * self.v_b[i] + (1 - self.beta2) * (db[i] ** 2)

                # bias correction
                m_w_hat = self.m_w[i] / (1 - self.beta1 ** (self.t + 1))
                m_b_hat = self.m_b[i] / (1 - self.beta1 ** (self.t + 1))
                v_w_hat = self.v_w[i] / (1 - self.beta2 ** (self.t + 1))
                v_b_hat = self.v_b[i] / (1 - self.beta2 ** (self.t + 1))

                grad_term_w = self.beta1 * m_w_hat + (1 - self.beta1) * dw[i] / (1 - self.beta1 ** (self.t + 1))
                grad_term_b = self.beta1 * m_b_hat + (1 - self.beta1) * db[i] / (1 - self.beta1 ** (self.t + 1))

                self.weights[i] -= self.lr * grad_term_w / (np.sqrt(v_w_hat) + self.epsilon)
                self.biases[i]  -= self.lr * grad_term_b / (np.sqrt(v_b_hat) + self.epsilon)

    # Training Step  with "Nesterov"
    def _train_step(self, x_batch, y_batch):
        if self.optimizer == "nesterov":
            # to look-ahead: w_look = w - momentum * v
            for i in range(self.layer_n - 1):
                self.weights[i] -= self.lr*self.momentum * self.v_w[i]
                self.biases[i]  -= self.lr*self.momentum * self.v_b[i]

            # Forward at the look-ahead position
            self.forward_pass(x_batch)
            out = self.h[-1]
            l2_norm_weights = 0
            for i in range(len(self.weights)):
                l2_norm_weights += np.sum(self.weights[i] ** 2)
            # for i in range(len(self.biases)):
            #     l2_norm_bias += np.sum(self.biases[i] ** 2)
                    
            l2_norm_params = l2_norm_weights #+ l2_norm_bias

            if self.loss_func=="cross_entropy":
                loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params # (1e-10) to prevent underflow
            elif self.loss_func=="sq_error" :
                loss= 0.5 * np.mean(np.sum((out - y_batch)**2, axis=1))
            else:
                loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params
            
            
            #loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params # (1e-10) to prevent underflow
            #loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis=1))
            dW, dB = self.backward_prop(y_batch)

            # add weight decay here
            for i in range(self.layer_n - 1):
                dW[i] += self.weight_decay * self.weights[i]

            # backward at the look-ahead position (go back to w_t)
            for i in range(self.layer_n - 1):
                self.weights[i] += self.lr*self.momentum * self.v_w[i]
                self.biases[i]  += self.lr*self.momentum * self.v_b[i]

            # update velocity: u_t = momentum*u_{t-1} + dW
            for i in range(self.layer_n - 1):
                self.v_w[i] = self.momentum * self.v_w[i] + dW[i]
                self.v_b[i] = self.momentum * self.v_b[i] + dB[i]

            # final param update: w = w - lr*u_t
            for i in range(self.layer_n - 1):
                self.weights[i] -= self.lr * self.v_w[i]
                self.biases[i]  -= self.lr * self.v_b[i]

            return loss
        else:
            # Normal forward/back
            self.forward_pass(x_batch)
            out = self.h[-1]

            l2_norm_weights=0
            
            for i in range(len(self.weights)):
                l2_norm_weights += np.sum(self.weights[i] ** 2)
            # for i in range(len(self.biases)):
            #     l2_norm_bias += np.sum(self.biases[i] ** 2)
                    
            l2_norm_params = l2_norm_weights #+ l2_norm_bias

            if self.loss_func=="cross_entropy":
                loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params # (1e-10) to prevent underflow
            elif self.loss_func=="sq_error" :
                loss= 0.5 * np.mean(np.sum((out - y_batch)**2, axis=1))
            else:
                loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params    
            
            
             

            #loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis=1))
            dW, dB = self.backward_prop(y_batch)
            self._update_params(dW, dB)
            return loss

    # Outer Training Loop 
    def training(self, x_train, y_train, x_val, y_val, epochs, batch_size):
       
        for ep in range(epochs):
            idx = np.random.permutation(x_train.shape[0])
            x_train_shuff = x_train[idx]
            y_train_shuff = y_train[idx]
            n_batches = len(x_train) // batch_size
            epoch_loss = 0.0
            for b in range(n_batches):
                start = b * batch_size
                end = start + batch_size
                x_batch = x_train_shuff[start:end]
                y_batch = y_train_shuff[start:end]
                loss = self._train_step(x_batch, y_batch)
                epoch_loss += loss
            avg_loss = epoch_loss / n_batches

            # Validation

            preds = self.predict(x_val)
            val_labels = np.argmax(y_val, axis=1)
            val_acc = np.mean(preds == val_labels)

            val_outputs = self.forward_pass(x_val)[-1]
        
            # Cross-entropy loss for validation
            #val_loss = -np.mean(np.sum(y_val * np.log(val_outputs + 1e-10), axis=1))
            # training
            preds_train = self.predict(x_train)
            train_labels = np.argmax(y_train, axis=1)
            train_acc = np.mean(preds_train == train_labels)

            l2_norm_weights=0
            
            for i in range(len(self.weights)):
                l2_norm_weights += np.sum(self.weights[i] ** 2)
            # for i in range(len(self.biases)):
            #     l2_norm_bias += np.sum(self.biases[i] ** 2)
                    
            l2_norm_params = l2_norm_weights

            if self.loss_func=="cross_entropy":
                val_loss = -np.mean(np.sum(y_val * np.log(val_outputs + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params # (1e-10) to prevent underflow
            elif self.loss_func=="sq_error" :
                val_loss= 0.5 * np.mean(np.sum((val_outputs - y_val)**2, axis=1))
            else:
                val_loss = -np.mean(np.sum(y_val * np.log(val_outputs + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params

            # Log metrics to wandb
            wandb.log({"epoch": ep+1, "training_loss": avg_loss, "validation_accuracy": val_acc, "training accuracy": train_acc "validation loss": val_loss})
            print(f"Epoch {ep+1}/{epochs} - loss={avg_loss:.4f}, val_acc={val_acc:.4f}, val_loss={val_loss}" )

    #Prediction 
    def predict(self, X):
        self.forward_pass(X)
        return np.argmax(self.h[-1], axis=1)




# train_sweep() function

def train_sweep():
    # Initialize wandb
    wandb.init()
    config = wandb.config

    #custom run name from hyperparameters

    run_name = f"hl_{config.num_layers}_hs_{config.hidden_size}_bs_{config.batch_size}_ac_{config.activation}_opt_{config.optimizer}"
    wandb.run.name = run_name

    if config["dataset"] == "fashion_mnist":
        (x_train_full, y_train_full), (x_test, y_test) = fashion_mnist.load_data()
    else:  # "mnist"
        (x_train_full, y_train_full), (x_test, y_test) = mnist.load_data()

    x_train_full = x_train_full.reshape(x_train_full.shape[0], -1) / 255.0
    x_test = x_test.reshape(x_test.shape[0], -1) / 255.0

    np.random.seed(42)
    idx = np.arange(x_train_full.shape[0])
    np.random.shuffle(idx)
    x_train_full = x_train_full[idx]
    y_train_full = y_train_full[idx]

    # 90% train, 10% validation
    train_size = int(0.9 * len(x_train_full))
    x_train, y_train = x_train_full[:train_size], y_train_full[:train_size]
    x_val, y_val = x_train_full[train_size:], y_train_full[train_size:]

    num_classes = 10
    y_train_1h = np.eye(num_classes)[y_train]
    y_val_1h = np.eye(num_classes)[y_val]
    y_test_1h = np.eye(num_classes)[y_test]

    # Build model
    # layers = [784] + [hidden_size] * num_layers + [10]
    model = feed_forward_NN_8(
        layers=[784] + [config["hidden_size"]] * config["num_layers"] + [10],
        optimizer=config["optimizer"],
        learning_rate=config["learning_rate"],
        momentum=config["momentum"],
        beta1=config["beta1"],
        beta2=config["beta2"],
        beta=config["beta"],
        epsilon=config["epsilon"],
        weight_decay=config["weight_decay"],
        weight_init=config["weight_init"],
        activation=config["activation"],
        loss=config["loss"],
    )

    # Train
    model.training(
        x_train=x_train,
        y_train=y_train_1h,
        x_val=x_val,
        y_val=y_val_1h,
        epochs=config["epochs"],
        batch_size=config["batch_size"],
    )

    #Evaluation on test set
    test_preds = model.predict(x_test)
    test_labels = np.argmax(y_test_1h, axis=1)
    test_acc = np.mean(test_preds == test_labels)
    
    wandb.log({"test_accuracy": test_acc})
    print("test accuracy ",test_acc)


# sweep configuration
sweep_config = {
    "method": "random", 
    "metric": {
        "name": "validation_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "epochs": {"values": [5,10]},
        "num_layers": {"values": [3,4,5]},
        "hidden_size": {"values": [32,16,64]},
        "weight_decay": {"values": [0.0, 0.0005, 0.5]},
        "learning_rate": {"values": [1e-3, 1e-4]},
        "optimizer": {"values": ["sgd", "momentum", "nag", "rmsprop", "adam", "nadam"]},
        "batch_size": {"values": [32,64,128]},
        "init_type": {"values": ["random","xavier"]},
        "activation": {"values": [ "tanh", "relu"]},
        "momentum": {"values": [0.9]},
        "beta1": {"values": [0.9]},
        "beta2": {"values": [0.999]},
        "beta_rms": {"values": [0.9]},
        "epsilon": {"values": [1e-8]},
        "loss_func":{"values":["cross_entropy"]}
    }
}


# Running the sweep



if __name__ == "__main__":
    # Creating sweep
    sweep_id = wandb.sweep(sweep_config, project="q4_sweep_project")
    # Launching sweep agent
    wandb.agent(sweep_id, function=train_sweep)





wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: g2aoqjdt
Sweep URL: https://wandb.ai/ed24s401-indian-institute-of-technology-madras/q4_sweep_project/sweeps/g2aoqjdt


wandb: Agent Starting Run: phf8zdp6 with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	beta1: 0.9
wandb: 	beta2: 0.999
wandb: 	beta_rms: 0.9
wandb: 	epochs: 15
wandb: 	epsilon: 1e-08
wandb: 	hidden_size: 32
wandb: 	init_type: random
wandb: 	learning_rate: 0.0001
wandb: 	loss_func: sq_error
wandb: 	momentum: 0.9
wandb: 	num_hidden_layers: 3
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0
wandb: Currently logged in as: ed24s401 (ed24s401-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/15 - loss=0.2674, val_acc=0.7638, val_loss=0.1743751192722
Epoch 2/15 - loss=0.1500, val_acc=0.8127, val_loss=0.14090028356791603
Epoch 3/15 - loss=0.1293, val_acc=0.8278, val_loss=0.12823391681726123
Epoch 4/15 - loss=0.1195, val_acc=0.8408, val_loss=0.11902486080913699
Epoch 5/15 - loss=0.1134, val_acc=0.8455, val_loss=0.11426869310717432
Epoch 6/15 - loss=0.1090, val_acc=0.8462, val_loss=0.11188272051937002
Epoch 7/15 - loss=0.1058, val_acc=0.8507, val_loss=0.11078548537210581
Epoch 8/15 - loss=0.1033, val_acc=0.8555, val_loss=0.10661031260267315
Epoch 9/15 - loss=0.1010, val_acc=0.8540, val_loss=0.10798276465733196
Epoch 10/15 - loss=0.0991, val_acc=0.8595, val_loss=0.10309541399735062
Epoch 11/15 - loss=0.0971, val_acc=0.8573, val_loss=0.10286096169843634
Epoch 12/15 - loss=0.0955, val_acc=0.8622, val_loss=0.10150745408797428
Epoch 13/15 - loss=0.0941, val_acc=0.8645, val_loss=0.09836587726005193
Epoch 14/15 - loss=0.0928, val_acc=0.8658, val_loss=0.09854203803645092
Epoch

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
test_accuracy,▁
training_loss,█▃▃▂▂▂▂▁▁▁▁▁▁▁▁
validation loss,█▅▄▃▂▂▂▂▂▁▁▁▁▁▁
validation_accuracy,▁▄▅▆▇▇▇▇▇█▇███▇
epoch,15
test_accuracy,0.8513
training_loss,0.09155
validation loss,0.10188
validation_accuracy,0.85783


...

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
test_accuracy,▁
training_loss,█▄▃▃▂▂▂▂▂▂▁▁▁▁▁
validation loss,█▅▄▄▂▃▇▂▄▃▂▂▁▂▃
validation_accuracy,▁▄▅▅▆▆▃▇▅▆▆██▇▆
epoch,15
test_accuracy,0.8634
training_loss,0.27677
validation loss,0.35515
validation_accuracy,0.87


wandb: Agent Starting Run: m08trzzk with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	beta1: 0.9
wandb: 	beta2: 0.999
wandb: 	beta_rms: 0.9
wandb: 	epochs: 15
wandb: 	epsilon: 1e-08
wandb: 	hidden_size: 32
wandb: 	init_type: random
wandb: 	learning_rate: 0.001
wandb: 	loss_func: sq_error
wandb: 	momentum: 0.9
wandb: 	num_hidden_layers: 3
wandb: 	optimizer: adam
wandb: 	weight_decay: 0


Epoch 1/15 - loss=0.1376, val_acc=0.8423, val_loss=0.11112433564752147
Epoch 2/15 - loss=0.1025, val_acc=0.8477, val_loss=0.10777411884002344
Epoch 3/15 - loss=0.0945, val_acc=0.8620, val_loss=0.09892814536193184
Epoch 4/15 - loss=0.0897, val_acc=0.8653, val_loss=0.09418991352338693
Epoch 5/15 - loss=0.0857, val_acc=0.8710, val_loss=0.09264558895498605
Epoch 6/15 - loss=0.0821, val_acc=0.8667, val_loss=0.09416271172475506
Epoch 7/15 - loss=0.0797, val_acc=0.8743, val_loss=0.08965335280493393
Epoch 8/15 - loss=0.0772, val_acc=0.8610, val_loss=0.09858342616893281
Epoch 9/15 - loss=0.0767, val_acc=0.8818, val_loss=0.08504300356954755
Epoch 10/15 - loss=0.0739, val_acc=0.8742, val_loss=0.08980353038586741
Epoch 11/15 - loss=0.0732, val_acc=0.8777, val_loss=0.08927301549656505
Epoch 12/15 - loss=0.0709, val_acc=0.8768, val_loss=0.08852451372980254
Epoch 13/15 - loss=0.0700, val_acc=0.8782, val_loss=0.08840949088910666
Epoch 14/15 - loss=0.0688, val_acc=0.8818, val_loss=0.08668242368013991
E

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
test_accuracy,▁
training_loss,█▄▄▃▃▂▂▂▂▂▂▁▁▁▁
validation loss,█▇▅▃▃▃▂▅▁▂▂▂▂▁▃
validation_accuracy,▁▂▄▅▆▅▇▄█▇▇▇▇█▆
epoch,15
test_accuracy,0.868
training_loss,0.06824
validation loss,0.09068
validation_accuracy,0.87167


wandb: Ctrl + C detected. Stopping sweep.


In [4]:
import numpy as np
import wandb
from keras.datasets import fashion_mnist


# Neural Network Class: feed_forward_NN_4

class feed_forward_NN_8b:
    def __init__(self,
                 layers,
                 optimizer,
                 learning_rate,
                 momentum,
                 beta1,
                 beta2,
                 beta,
                 epsilon,
                 weight_decay,
                 init_type,
                 activation,
                 loss_func
                 ):
    
        
        self.layers = layers
        self.layer_n = len(layers)
        self.optimizer = optimizer.lower()
        self.lr = learning_rate
        self.momentum = momentum
        self.beta1 = beta1
        self.beta2 = beta2
        self.beta= beta
        self.epsilon = epsilon
        self.weight_decay = weight_decay
        self.init_type = init_type.lower()
        self.activation = activation.lower()
        self.loss_func=loss_func.lower()

        # Initialize Weights & BiaseS
        self.weights = []
        self.biases = []
        for i in range(self.layer_n - 1):
            if self.init_type == "xavier":
                # "Xavier" initialization
                w = np.random.randn(layers[i], layers[i+1]) * np.sqrt(1.0 / layers[i])
            else:
                # "random" initialization
                w = np.random.randn(layers[i], layers[i+1]) * np.sqrt(2.0 / layers[i])
            b = np.zeros((1, layers[i+1]))
            self.weights.append(w)
            self.biases.append(b)

        # initialize extra Params 
        if self.optimizer in ["momentum", "nesterov", "rmsprop", "adam", "nadam"]:
            self.v_w = [np.zeros_like(w) for w in self.weights]
            self.v_b = [np.zeros_like(b) for b in self.biases]
        if self.optimizer in ["adam", "nadam"]:
            self.m_w = [np.zeros_like(w) for w in self.weights]
            self.m_b = [np.zeros_like(b) for b in self.biases]
            self.t = 0

    # activations 
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def tanh(self, x):
        return np.tanh(x)
    
    def relu(self, x):
        return np.maximum(0, x)

    def activate(self, x):
        if self.activation == "sigmoid":
            return self.sigmoid(x)
        elif self.activation == "tanh":
            return self.tanh(x)
        elif self.activation == "relu":
            return self.relu(x)
        else:
            return self.sigmoid(x) 
        
    # derivatives
    def derivative(self, a):

        if self.activation == "sigmoid":
            return a * (1 - a)
        elif self.activation == "tanh":
            return 1 - a**2
        elif self.activation == "relu":
            return (a > 0).astype(float)
        else:
            return a * (1 - a) 

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    # Forward Pass
    def forward_pass(self, x):
        self.h = [x]  
        # Hidden layers
        for i in range(self.layer_n - 2):
            z = np.dot(self.h[i], self.weights[i]) + self.biases[i]
            act = self.activate(z)
            self.h.append(act)
        # Output layer- softmax
        z_out = np.dot(self.h[-1], self.weights[-1]) + self.biases[-1]
        out = self.softmax(z_out)
        self.h.append(out)
        return self.h

    # Backward Pass
    def backward_prop(self, y_true):
        m = y_true.shape[0]
        dw = [None] * (self.layer_n - 1)
        db = [None] * (self.layer_n - 1)

        # Cross-entropy derivative for output layer
        if self.loss_func=="cross_entropy":
            delta = self.h[-1] - y_true  # shape: (batch_size, output_dim)
        elif self.loss_func=="sq_error":
            batch_size_sq=len(self.h[-1])
            classes_sq=len(self.h[-1][0])
            delta=np.zeros((batch_size_sq,classes_sq))

            for i in range(batch_size_sq):
                jacobian_softmax= np.diag(self.h[-1][i]) - np.outer(self.h[-1][i], self.h[-1][i])
                # print(jacobian_softmax.shape)
                # print(self.h[-1][i])    
                delta[i]= 2*np.dot(self.h[-1][i]-y_true[i], jacobian_softmax)
                
        else:
            delta = self.h[-1] - y_true 


        # Propagation
        for i in reversed(range(self.layer_n - 1)):
            dw[i] = np.dot(self.h[i].T, delta) / m
            db[i] = np.sum(delta, axis=0, keepdims=True) / m
            if i > 0:
                # For hidden layers, multiply by derivative of activation
                delta = np.dot(delta, self.weights[i].T) * self.derivative(self.h[i])
        return dw, db

    # Param Updates for "Non-Nesterov" 
    def _update_params(self, dw, db):
        # Add weight decay to each gradient
        for i in range(self.layer_n - 1):
            dw[i] += self.weight_decay * self.weights[i]

        if self.optimizer == "sgd":
            for i in range(self.layer_n - 1):
                self.weights[i] -= self.lr * dw[i]
                self.biases[i] -= self.lr * db[i]

        elif self.optimizer == "momentum":
            for i in range(self.layer_n - 1):
                self.v_w[i] = self.momentum * self.v_w[i] + dw[i]
                self.v_b[i] = self.momentum * self.v_b[i] + db[i]
                self.weights[i] -= self.lr * self.v_w[i]
                self.biases[i] -= self.lr * self.v_b[i]

        elif self.optimizer == "rmsprop":
            for i in range(self.layer_n - 1):
                self.v_w[i] = self.beta * self.v_w[i] + (1 - self.beta) * (dw[i] ** 2)
                self.v_b[i] = self.beta * self.v_b[i] + (1 - self.beta) * (db[i] ** 2)
                self.weights[i] -= self.lr * dw[i] / (np.sqrt(self.v_w[i]) + self.epsilon)
                self.biases[i]  -= self.lr * db[i] / (np.sqrt(self.v_b[i]) + self.epsilon)

        elif self.optimizer == "adam":
            self.t += 1
            for i in range(self.layer_n - 1):
                self.m_w[i] = self.beta1 * self.m_w[i] + (1 - self.beta1) * dw[i]
                self.m_b[i] = self.beta1 * self.m_b[i] + (1 - self.beta1) * db[i]
                self.v_w[i] = self.beta2 * self.v_w[i] + (1 - self.beta2) * (dw[i] ** 2)
                self.v_b[i] = self.beta2 * self.v_b[i] + (1 - self.beta2) * (db[i] ** 2)

                # bias correction
                m_w_hat = self.m_w[i] / (1 - self.beta1 ** self.t)
                m_b_hat = self.m_b[i] / (1 - self.beta1 ** self.t)
                v_w_hat = self.v_w[i] / (1 - self.beta2 ** self.t)
                v_b_hat = self.v_b[i] / (1 - self.beta2 ** self.t)

                self.weights[i] -= self.lr * m_w_hat / (np.sqrt(v_w_hat) + self.epsilon)
                self.biases[i]  -= self.lr * m_b_hat / (np.sqrt(v_b_hat) + self.epsilon)

        elif self.optimizer == "nadam":
            self.t += 1
            for i in range(self.layer_n - 1):
                self.m_w[i] = self.beta1 * self.m_w[i] + (1 - self.beta1) * dw[i]
                self.m_b[i] = self.beta1 * self.m_b[i] + (1 - self.beta1) * db[i]
                self.v_w[i] = self.beta2 * self.v_w[i] + (1 - self.beta2) * (dw[i] ** 2)
                self.v_b[i] = self.beta2 * self.v_b[i] + (1 - self.beta2) * (db[i] ** 2)

                # bias correction
                m_w_hat = self.m_w[i] / (1 - self.beta1 ** (self.t + 1))
                m_b_hat = self.m_b[i] / (1 - self.beta1 ** (self.t + 1))
                v_w_hat = self.v_w[i] / (1 - self.beta2 ** (self.t + 1))
                v_b_hat = self.v_b[i] / (1 - self.beta2 ** (self.t + 1))

                grad_term_w = self.beta1 * m_w_hat + (1 - self.beta1) * dw[i] / (1 - self.beta1 ** (self.t + 1))
                grad_term_b = self.beta1 * m_b_hat + (1 - self.beta1) * db[i] / (1 - self.beta1 ** (self.t + 1))

                self.weights[i] -= self.lr * grad_term_w / (np.sqrt(v_w_hat) + self.epsilon)
                self.biases[i]  -= self.lr * grad_term_b / (np.sqrt(v_b_hat) + self.epsilon)

    # Training Step  with "Nesterov"
    def _train_step(self, x_batch, y_batch):
        if self.optimizer == "nesterov":
            # to look-ahead: w_look = w - momentum * v
            for i in range(self.layer_n - 1):
                self.weights[i] -= self.lr*self.momentum * self.v_w[i]
                self.biases[i]  -= self.lr*self.momentum * self.v_b[i]

            # Forward at the look-ahead position
            self.forward_pass(x_batch)
            out = self.h[-1]
            l2_norm_weights = 0
            for i in range(len(self.weights)):
                l2_norm_weights += np.sum(self.weights[i] ** 2)
            # for i in range(len(self.biases)):
            #     l2_norm_bias += np.sum(self.biases[i] ** 2)
                    
            l2_norm_params = l2_norm_weights #+ l2_norm_bias

            if self.loss_func=="cross_entropy":
                loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params # (1e-10) to prevent underflow
            elif self.loss_func=="sq_error" :
                loss= 0.5 * np.mean(np.sum((out - y_batch)**2, axis=1))
            else:
                loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params
            
            
            #loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params # (1e-10) to prevent underflow
            #loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis=1))
            dW, dB = self.backward_prop(y_batch)

            # add weight decay here
            for i in range(self.layer_n - 1):
                dW[i] += self.weight_decay * self.weights[i]

            # backward at the look-ahead position (go back to w_t)
            for i in range(self.layer_n - 1):
                self.weights[i] += self.lr*self.momentum * self.v_w[i]
                self.biases[i]  += self.lr*self.momentum * self.v_b[i]

            # update velocity: u_t = momentum*u_{t-1} + dW
            for i in range(self.layer_n - 1):
                self.v_w[i] = self.momentum * self.v_w[i] + dW[i]
                self.v_b[i] = self.momentum * self.v_b[i] + dB[i]

            # final param update: w = w - lr*u_t
            for i in range(self.layer_n - 1):
                self.weights[i] -= self.lr * self.v_w[i]
                self.biases[i]  -= self.lr * self.v_b[i]

            return loss
        else:
            # Normal forward/back
            self.forward_pass(x_batch)
            out = self.h[-1]

            l2_norm_weights=0
            
            for i in range(len(self.weights)):
                l2_norm_weights += np.sum(self.weights[i] ** 2)
            # for i in range(len(self.biases)):
            #     l2_norm_bias += np.sum(self.biases[i] ** 2)
                    
            l2_norm_params = l2_norm_weights #+ l2_norm_bias

            if self.loss_func=="cross_entropy":
                loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params # (1e-10) to prevent underflow
            elif self.loss_func=="sq_error" :
                loss= 0.5 * np.mean(np.sum((out - y_batch)**2, axis=1))
            else:
                loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params    
            
            
             

            #loss = -np.mean(np.sum(y_batch * np.log(out + 1e-10), axis=1))
            dW, dB = self.backward_prop(y_batch)
            self._update_params(dW, dB)
            return loss

    # Outer Training Loop 
    def training(self, x_train, y_train, x_val, y_val, epochs, batch_size):
       
        for ep in range(epochs):
            idx = np.random.permutation(x_train.shape[0])
            x_train_shuff = x_train[idx]
            y_train_shuff = y_train[idx]
            n_batches = len(x_train) // batch_size
            epoch_loss = 0.0
            for b in range(n_batches):
                start = b * batch_size
                end = start + batch_size
                x_batch = x_train_shuff[start:end]
                y_batch = y_train_shuff[start:end]
                loss = self._train_step(x_batch, y_batch)
                epoch_loss += loss
            avg_loss = epoch_loss / n_batches

            # Validation

            preds = self.predict(x_val)
            val_labels = np.argmax(y_val, axis=1)
            val_acc = np.mean(preds == val_labels)

            val_outputs = self.forward_pass(x_val)[-1]
        
            # Cross-entropy loss for validation
            #val_loss = -np.mean(np.sum(y_val * np.log(val_outputs + 1e-10), axis=1))
            # training
            preds_train = self.predict(x_train)
            train_labels = np.argmax(y_train, axis=1)
            train_acc = np.mean(preds_train == train_labels)

            l2_norm_weights=0
            
            for i in range(len(self.weights)):
                l2_norm_weights += np.sum(self.weights[i] ** 2)
            # for i in range(len(self.biases)):
            #     l2_norm_bias += np.sum(self.biases[i] ** 2)
                    
            l2_norm_params = l2_norm_weights

            if self.loss_func=="cross_entropy":
                val_loss = -np.mean(np.sum(y_val * np.log(val_outputs + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params # (1e-10) to prevent underflow
            elif self.loss_func=="sq_error" :
                val_loss= 0.5 * np.mean(np.sum((val_outputs - y_val)**2, axis=1))
            else:
                val_loss = -np.mean(np.sum(y_val * np.log(val_outputs + 1e-10), axis = 1)) +  (self.weight_decay/2) * l2_norm_params

            # Log metrics to wandb
            wandb.log({"epoch": ep+1, "training_loss": avg_loss, "validation_accuracy": val_acc, "training accuracy": train_acc, "validation loss": val_loss})
            print(f"Epoch {ep+1}/{epochs} - loss={avg_loss:.4f}, val_acc={val_acc:.4f}, val_loss={val_loss}" )

    #Prediction 
    def predict(self, X):
        self.forward_pass(X)
        return np.argmax(self.h[-1], axis=1)




# train_sweep() function

def train_sweep():
    # Initialize wandb
    wandb.init()
    config = wandb.config

    #custom run name from hyperparameters

    run_name = f"hl_{config.num_layers}_hs_{config.hidden_size}_bs_{config.batch_size}_ac_{config.activation}_opt_{config.optimizer}"
    wandb.run.name = run_name

    if config["dataset"] == "fashion_mnist":
        (x_train_full, y_train_full), (x_test, y_test) = fashion_mnist.load_data()
    else:  # "mnist"
        (x_train_full, y_train_full), (x_test, y_test) = mnist.load_data()

    x_train_full = x_train_full.reshape(x_train_full.shape[0], -1) / 255.0
    x_test = x_test.reshape(x_test.shape[0], -1) / 255.0

    np.random.seed(42)
    idx = np.arange(x_train_full.shape[0])
    np.random.shuffle(idx)
    x_train_full = x_train_full[idx]
    y_train_full = y_train_full[idx]

    # 90% train, 10% validation
    train_size = int(0.9 * len(x_train_full))
    x_train, y_train = x_train_full[:train_size], y_train_full[:train_size]
    x_val, y_val = x_train_full[train_size:], y_train_full[train_size:]

    num_classes = 10
    y_train_1h = np.eye(num_classes)[y_train]
    y_val_1h = np.eye(num_classes)[y_val]
    y_test_1h = np.eye(num_classes)[y_test]

    # Build model
    # layers = [784] + [hidden_size] * num_layers + [10]
    model = feed_forward_NN_8b(
        layers=[784] + [config["hidden_size"]] * config["num_layers"] + [10],
        optimizer=config["optimizer"],
        learning_rate=config["learning_rate"],
        momentum=config["momentum"],
        beta1=config["beta1"],
        beta2=config["beta2"],
        beta=config["beta"],
        epsilon=config["epsilon"],
        weight_decay=config["weight_decay"],
        weight_init=config["weight_init"],
        activation=config["activation"],
        loss=config["loss"],
    )

    # Train
    model.training(
        x_train=x_train,
        y_train=y_train_1h,
        x_val=x_val,
        y_val=y_val_1h,
        epochs=config["epochs"],
        batch_size=config["batch_size"],
    )

    #Evaluation on test set
    test_preds = model.predict(x_test)
    test_labels = np.argmax(y_test_1h, axis=1)
    test_acc = np.mean(test_preds == test_labels)
    
    wandb.log({"test_accuracy": test_acc})
    print("test accuracy ",test_acc)


# sweep configuration
sweep_config = {
    "method": "random", 
    "metric": {
        "name": "validation_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "epochs": {"values": [5,10]},
        "num_layers": {"values": [3,4,5]},
        "hidden_size": {"values": [32,16,64]},
        "weight_decay": {"values": [0.0, 0.0005, 0.5]},
        "learning_rate": {"values": [1e-3, 1e-4]},
        "optimizer": {"values": ["sgd", "momentum", "nag", "rmsprop", "adam", "nadam"]},
        "batch_size": {"values": [32,64,128]},
        "init_type": {"values": ["random","xavier"]},
        "activation": {"values": [ "tanh", "relu","sigmoid"]},
        "momentum": {"values": [0.9]},
        "beta1": {"values": [0.9]},
        "beta2": {"values": [0.999]},
        "beta_rms": {"values": [0.9]},
        "epsilon": {"values": [1e-8]},
        "loss_func":{"values":["cross_entropy"]},
        "dataset":{"values":["fashion_mnist"]}
    }
}


# Running the sweep



if __name__ == "__main__":
    # Creating sweep
    sweep_id = wandb.sweep(sweep_config, project="q4_sweep_project")
    # Launching sweep agent
    wandb.agent(sweep_id, function=train_sweep)





Create sweep with ID: jl0s4l1o
Sweep URL: https://wandb.ai/ed24s401-indian-institute-of-technology-madras/q4_sweep_project/sweeps/jl0s4l1o


wandb: Network error (SSLError), entering retry loop.
wandb: Agent Starting Run: z4mu7ngb with config:
wandb: 	activation: relu
wandb: 	batch_size: 128
wandb: 	beta1: 0.9
wandb: 	beta2: 0.999
wandb: 	beta_rms: 0.9
wandb: 	dataset: fashion_mnist
wandb: 	epochs: 10
wandb: 	epsilon: 1e-08
wandb: 	hidden_size: 64
wandb: 	init_type: random
wandb: 	learning_rate: 0.0001
wandb: 	loss_func: cross_entropy
wandb: 	momentum: 0.9
wandb: 	num_layers: 4
wandb: 	optimizer: momentum
wandb: 	weight_decay: 0.5


Run z4mu7ngb errored:
Traceback (most recent call last):
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
    beta=config["beta"],
         ~~~~~~^^^^^^^^
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_config.py", line 130, in __getitem__
    return self._items[key]
           ~~~~~~~~~~~^^^^^
KeyError: 'beta'

wandb: ERROR Run z4mu7ngb errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
wandb: ERROR     beta=config["beta"],
wandb: ERROR          ~~~~~~^^^^^^^

Run vgq4ktau errored:
Traceback (most recent call last):
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
    beta=config["beta"],
         ~~~~~~^^^^^^^^
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_config.py", line 130, in __getitem__
    return self._items[key]
           ~~~~~~~~~~~^^^^^
KeyError: 'beta'

wandb: ERROR Run vgq4ktau errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
wandb: ERROR     beta=config["beta"],
wandb: ERROR          ~~~~~~^^^^^^^

Run uyv586k2 errored:
Traceback (most recent call last):
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
    beta=config["beta"],
         ~~~~~~^^^^^^^^
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_config.py", line 130, in __getitem__
    return self._items[key]
           ~~~~~~~~~~~^^^^^
KeyError: 'beta'

wandb: ERROR Run uyv586k2 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
wandb: ERROR     beta=config["beta"],
wandb: ERROR          ~~~~~~^^^^^^^

Run 0pekahxd errored:
Traceback (most recent call last):
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
    beta=config["beta"],
         ~~~~~~^^^^^^^^
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_config.py", line 130, in __getitem__
    return self._items[key]
           ~~~~~~~~~~~^^^^^
KeyError: 'beta'

wandb: ERROR Run 0pekahxd errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
wandb: ERROR     beta=config["beta"],
wandb: ERROR          ~~~~~~^^^^^^^

Run eqjwuii7 errored:
Traceback (most recent call last):
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
    beta=config["beta"],
         ~~~~~~^^^^^^^^
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_config.py", line 130, in __getitem__
    return self._items[key]
           ~~~~~~~~~~~^^^^^
KeyError: 'beta'

wandb: ERROR Run eqjwuii7 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
wandb: ERROR     beta=config["beta"],
wandb: ERROR          ~~~~~~^^^^^^^

Run wfx4nk6n errored:
Traceback (most recent call last):
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
    beta=config["beta"],
         ~~~~~~^^^^^^^^
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_config.py", line 130, in __getitem__
    return self._items[key]
           ~~~~~~~~~~~^^^^^
KeyError: 'beta'

wandb: ERROR Run wfx4nk6n errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "C:\Users\acer\AppData\Local\Temp\ipykernel_24364\153520030.py", line 393, in train_sweep
wandb: ERROR     beta=config["beta"],
wandb: ERROR          ~~~~~~^^^^^^^